# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Li  ->  S. Li  |  ['S. Li']
N. Pulatova  ->  N. Pulatova  |  ['N. Pulatova']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 66 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.02682


extracting tarball to tmp_2406.02682...

 done.


J. Olofsson  ->  J. Olofsson  |  ['J. Olofsson']
T. Henning  ->  T. Henning  |  ['T. Henning']


...

extracting tarball to tmp_2406.03410... done.
Retrieving document from  https://arxiv.org/e-print/2406.03490


extracting tarball to tmp_2406.03490...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure day_EQ_revised.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:704: LatexWarning: Could not find graphic \includegraphics[scale=0.51]{day_EQ_revised.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 118 bibliographic references in tmp_2406.03490/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.02682-b31b1b.svg)](https://arxiv.org/abs/2406.02682) | **The near-infrared degree of polarization in debris disks. Toward a self-consistent approach to model scattered light observations**  |
|| <mark>J. Olofsson</mark>, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-06-06*|
|*Comments*| *Accepted for publication in A&A, abstract shortened*|
|**Abstract**|            Debris disks give us the unique opportunity to probe the properties of small $\mu$m-sized particles, allowing us to peer into the constituents of their parent bodies, young analogs of comets and asteroids of our solar system. In the past, studies of the total intensity phase function have proven powerful to constrain the main characteristics of the dust particles in debris disks. Nonetheless, there can remain some degeneracies in the modeling that can be alleviated when considering polarized intensity observations. We obtained new near-IR scattered light observations of four young debris disks which allow us to constrain the degree of linear polarization as a function of the scattering angle. All four debris disks are detected in polarized intensity, and three are also recovered in total intensity. We measured peak degree of polarization of $\lesssim 40$\% for all three disks. We find that the particles must consist of highly refractive and absorbing material. For HD129590, by measuring the polarization fraction beyond the birth ring, we constrain the width of the size distribution to be smaller and smaller, compatible with the effect of radiation pressure. We put these findings to the test and present a self-consistent approach to produce synthetic images, assuming different profiles for the radiation pressure strength, and accounting for the presence of unbound grains. We find the contribution of these grains to be especially critical to reproduce the increasing degree of polarization with stellocentric distances. Some of our results might seem difficult to reconcile with our understanding of cosmic dust but since similar results have been obtained for other disks, we discuss the current limitation of available light scattering models as well as possible avenues to alleviate these unfortunate limitations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.03254-b31b1b.svg)](https://arxiv.org/abs/2406.03254) | **What excites the optical emission in X-ray-selected galaxies?**  |
|| <mark>N. Pulatova</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>R. Seeburger</mark>, <mark>M. Demianenko</mark> |
|*Appeared on*| *2024-06-06*|
|*Comments*| *12 pages, 13 figures*|
|**Abstract**|            We present a study of $1347$ galaxies at $z<0.35$ with detected nuclear X-ray emission and optical emission line diagnostics in the Baldwin-Phillips-Terlevich (BPT) diagram. This sample was obtained by cross-matching the X-ray Multi-Mirror Mission Observatory - Newton (XMM-Newton) DR10 catalogue with Sloan Digital Sky Survey (SDSS) DR17 galaxies with well-measured line ratios. The distribution of these sources in the BPT diagram covers all three excitation regimes: Ionized Hydrogen (HII) regions (23\%), `composites' (30\%), and Seyfert galaxies with the low ionization nuclear emission line regions (LINERs) (47\%). In contrast, the fraction of objects classified as active galactic nuclei (AGN) in the SDSS subsample selected for cross-match with XMM-Newton is only 13\%. This fact illustrates that X-ray emission from galaxies commonly points towards the presence of AGN. Our data show, for the first time, a clear dependence of the BPT position on the ratio of the X-ray to $H\alpha$ fluxes. Sources dominated by X-ray emission lie in the Seyfert and LINER regimes of the BPT diagram. Most sources with a low X-ray-to-$H\alpha$-luminosity ratio, $log_{10}(L_X/L_{H\alpha}) < 1.0$, lie in the HII regime. In our sample, there are even 45 galaxies that have $L^{Star}_{XR}/L^{Total}_{Xray}>0.5$. In contrast, the positions of the sample members in the BPT diagram exhibit {no} dependence on the X-ray hardness ratio. Our finding suggests that the X-ray-to-$H\alpha$ ratio can help us to differentiate galaxies whose X-ray flux is dominated by an AGN {from galaxies with} central X-ray binaries and other stellar X-ray sources.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.03490-b31b1b.svg)](https://arxiv.org/abs/2406.03490) | **Simultaneous retrieval of orbital phase resolved JWST/MIRI emission spectra of the hot Jupiter WASP-43b: evidence of water, ammonia and carbon monoxide**  |
|| J. Yang, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-06-06*|
|*Comments*| *17 pages, 14 figures, accepted for publication in MNRAS. Comments welcome!*|
|**Abstract**|            Spectroscopic phase curves of hot Jupiters measure their emission spectra at multiple orbital phases, thus enabling detailed characterisation of their atmospheres. Precise constraints on the atmospheric composition of these exoplanets offer insights into their formation and evolution. We analyse four phase-resolved emission spectra of the hot Jupiter WASP-43b, generated from a phase curve observed with the MIRI/LRS onboard the JWST, to retrieve its atmospheric properties. Using a parametric 2D temperature model and assuming a chemically homogeneous atmosphere within the observed pressure region, we simultaneously fit the four spectra to constrain the abundances of atmospheric constituents, thereby yielding more precise constraints than previous work that analysed each spectrum independently. Our analysis reveals statistically significant evidence of NH3 (4$\sigma$) in a hot Jupiter's emission spectra for the first time, along with evidence of H2O (6.5$\sigma$), CO (3.1$\sigma$), and a non-detection of CH4. With our abundance constraints, we tentatively estimate the metallicity of WASP-43b at 0.6-6.5$\times$solar and its C/O ratio at 0.6-0.9. Our findings offer vital insights into the atmospheric conditions and formation history of WASP-43b by simultaneously constraining the abundances of carbon, oxygen, and nitrogen-bearing species.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.03308-b31b1b.svg)](https://arxiv.org/abs/2406.03308) | **How Gaia sheds light on the Milky Way star cluster population**  |
|| <mark>T. Cantat-Gaudin</mark>, L. Casamiquela |
|*Appeared on*| *2024-06-06*|
|*Comments*| *Accepted review article for New Astronomy Reviews (as part of a Special Issue: "Gaia, the first crop of discoveries")*|
|**Abstract**|            Star clusters are among the first celestial objects catalogued by early astronomers. As simple and coeval populations, their study has been instrumental in charting the properties of the Milky Way and providing insight into stellar evolution through the 20th century. Clusters were traditionally spotted as local stellar overdensities in the plane of the sky. In recent decades, for a limited number of nearby clusters, it became possible to identify cluster members through their clustering in proper motion space. With its astrometric data of unprecedented precision, the Gaia mission has completely revolutionised our ability to discover and characterise Milky Way star clusters, to map their large-scale distribution, and to investigate their internal structure. In this review we focus on the population of open clusters, residing in the Galactic disc. We summarise the current state of the Gaia-updated cluster census and studies of young clusters and associations. We discuss recent developments in techniques for cluster detection and age estimation. We also review results enabled by Gaia data concerning the dynamical evolution of gravitationally bound clusters and their stellar inventory.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.02946-b31b1b.svg)](https://arxiv.org/abs/2406.02946) | **CAMEL. II. A 3D Coronal Mass Ejection Catalog Based on Coronal Mass Ejection Automatic Detection with Deep Learning**  |
|| J. Shan, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-06-06*|
|*Comments*| **|
|**Abstract**|            Coronal mass ejections (CMEs) are major drivers of geomagnetic storms, which may cause severe space weather effects. Automating the detection, tracking, and three-dimensional (3D) reconstruction of CMEs is important for operational predictions of CME arrivals. The COR1 coronagraphs on board the Solar Terrestrial Relations Observatory spacecraft have facilitated extensive polarization observations, which are very suitable for the establishment of a 3D CME system. We have developed such a 3D system comprising four modules: classification, segmentation, tracking, and 3D reconstructions. We generalize our previously pretrained classification model to classify COR1 coronagraph images. Subsequently, as there are no publicly available CME segmentation data sets, we manually annotate the structural regions of CMEs using Large Angle and Spectrometric Coronagraph C2 observations. Leveraging transformer-based models, we achieve state-of-the-art results in CME segmentation. Furthermore, we improve the tracking algorithm to solve the difficult separation task of multiple CMEs. In the final module, tracking results, combined with the polarization ratio technique are used to develop the first single-view 3D CME catalog without requiring manual mask annotation. Our method provides higher precision in automatic 2D CME catalog and more reliable physical parameters of CMEs, including 3D propagation direction and speed. The aforementioned 3D CME system can be applied to any coronagraph data with the capability of polarization measurements.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.03320-b31b1b.svg)](https://arxiv.org/abs/2406.03320) | **Detection of extended gamma-ray emission in the vicinity of Cl Danks 1 and 2**  |
|| <mark>J. Liu</mark>, B. Liu, R. Yang |
|*Appeared on*| *2024-06-06*|
|*Comments*| *9 pages, 4 figures, submitted to APJL*|
|**Abstract**|            We report the detection of high-energy gamma-ray emission towards the G305 star-forming region. Using almost 15 years of observation data from {\sl Fermi} Large Area Telescope, we detected an extended gamma-ray source in this region with a significance of $\sim 13 \sigma$. The gamma-ray radiation reveals a clear pion-bump feature and can be fitted with the power law parent proton spectrum with an index of $-2.5$. The total cosmic ray (CR) proton energy in the gamma-ray production region is estimated to be the order of $10^{49}\ \rm erg$. We further derived the CR radial distribution from both the gamma-ray emission and gas distribution and found it roughly obeys the $1/r$ type profile, which is consistent with other similar systems and expected from the continuous injection of CRs by the central powerful young massive star cluster Danks 1 or Danks 2 in this region. Together with former detections of similar gamma-ray structures, such as Cygnus cocoon, Westerlund 1, Westerlund 2, NGC 3603, and W40, the detection supports the hypothesis that young massive star clusters are CR accelerators.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.03410-b31b1b.svg)](https://arxiv.org/abs/2406.03410) | **Probing the distinct extinction law of the Pillars of Creation in M16 with JWST**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2024-06-06*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Letters (9 pages, 4 figures, 2 tables)*|
|**Abstract**|            Investigating the extinction law in regions of high dust extinction, such as the Pillars of Creation within the M16 region, is crucial for understanding the densest parts of the interstellar medium (ISM). In this study, we utilize observations from the Near-Infrared Camera (NIRCam) and the Mid-Infrared Instrument (MIRI) onboard the James Webb Space Telescope (JWST) to analyze the color-excess ratios $E(F090W-\lambda)/E(F090W-F200W)$ across a wavelength range of $0.9-7.7\,\mu\mathrm{m}$. Our method involves performing linear regression on color-color diagrams to derive these ratios. The enhanced detection capabilities of JWST data allow us to probe the distinct extinction law to the densest regions in M16 corresponding to an extinction depth up to $A_V \sim 60$\,mag. Remarkably, the resultant color-excess ratio curve exhibits a flatter profile than predicted by typical dust extinction models with $R_V = 5.5$ for dense ISM environments. Moreover, we observe that the mid-infrared (MIR) extinction law diverges from the near-infrared (NIR) power-law, showing a tendency for the slope to flatten as the wavelength increases. These findings have significant implications for our understanding of the dust properties in dense interstellar environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2406.02682.md
    + _build/html/tmp_2406.02682/./HD191089_full.png
    + _build/html/tmp_2406.02682/./HD157587_full.png
    + _build/html/tmp_2406.02682/./HD115600_full.png
    + _build/html/tmp_2406.02682/./HD129590_full.png
    + _build/html/tmp_2406.02682/./tau_cs.png
    + _build/html/tmp_2406.02682/./HD157587_pfunc.png
exported in  _build/html/2406.03254.md
    + _build/html/tmp_2406.03254/./DR-17-X-ray-H_alpha_Lum.jpg
    + _build/html/tmp_2406.03254/./DR-17-BPT_sdss-z-X-Ha-1.jpg
    + _build/html/tmp_2406.03254/./DR17-X-ray-RA-DEC.png
exported in  _build/html/2406.03490.md
    + _build/html/tmp_2406.03490/./Figures/compare_spectra_opacity_1500K_revised.png
    + _build/html/tmp_2406.03490/./Figures/model.png
    + _build/html/tmp_2406.03490/./Figures/TP_model_revised.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# The near-infrared degree of polarization in debris disks.$\thanks{Based on observations made with ESO Telescopes at the Paranal Observatory under programs ID 105.20GP.001 and 109.237K.001. The fits files of the observations are available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.02682-b31b1b.svg)](https://arxiv.org/abs/2406.02682)<mark>Appeared on: 2024-06-06</mark> -  _Accepted for publication in A&A, abstract shortened_

</div>
<div id="authors">

<mark>J. Olofsson</mark>, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Debris disks give us the unique opportunity to probe the properties of small $\mu$ m-sized particles, allowing us to peer into the constituents of their parent bodies, young analogs of comets and asteroids of our solar system. In the past, studies of the total intensity phase function, the brightness of the disk as a function of the scattering angle, have proven powerful to constrain the main characteristics of the dust particles in debris disks. Nonetheless, there can remain some degeneracies in the modeling that can be alleviated when considering polarized intensity observations. We obtained new near-infrared scattered light observations of four young debris disks, and used state-of-the-art algorithms to recover the total intensity and linear polarimetric images of the disks. These images allow us to constrain the degree of linear polarization as a function of the scattering angle. All four debris disks are detected in polarized intensity, and three are also recovered in total intensity. We measured peak degree of polarization of $\lesssim 40$ \% for all three disks. For the disk around HD 129590, we are furthermore able to determine the degree of polarization in the radiation pressure driven halo. To reproduce the observed polarization fractions, we find that the particles must consist of highly refractive and absorbing material. For HD 129590, by measuring the polarization fraction beyond the birth ring, we constrain the width of the size distribution to be smaller and smaller, compatible with the effect of radiation pressure. We put these findings to the test and present a self-consistent approach to produce synthetic images, assuming different profiles for the radiation pressure strength, and accounting for the presence of unbound grains. We find the contribution of these grains to be especially critical to reproduce the increasing degree of polarization with stellocentric distances. Some of our results (namely a very small blow-out size and very large $(n,k)$ values for the optical constants) required to reproduce the observed degree of polarization might seem difficult to reconcile with our understanding of cosmic dust. Similar results have been obtained for other disks and we discuss the current limitation of available light scattering models as well as possible avenues to alleviate these unfortunate limitations.

</div>

<div id="div_fig1">

<img src="tmp_2406.02682/./HD191089_full.png" alt="Fig4.1" width="25%"/><img src="tmp_2406.02682/./HD157587_full.png" alt="Fig4.2" width="25%"/><img src="tmp_2406.02682/./HD115600_full.png" alt="Fig4.3" width="25%"/><img src="tmp_2406.02682/./HD129590_full.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** _Top to bottom:_ observations of HD 191089, HD 157587, HD 115600, and HD 129590. _Left:_ polarimetry, _center:_ total intensity, _right:_ degree of polarization (polarimetry over total intensity). The scaling is linear for the right column and between $0$ and $1$, while in square root for the left and center columns. On the left panel, the horizontal bar represents $1$\arcsec$$ and the distance in au is reported below. North is up, East is right, as indicated by the compass on the leftmost panels. (*fig:full*)

</div>
<div id="div_fig2">

<img src="tmp_2406.02682/./tau_cs.png" alt="Fig7" width="100%"/>

**Figure 7. -** _Top to bottom:_ diagnostics for Runs 1 to 4, respectively. _Left:_ optical depth profile as a function of the stellocentric distance. The dashed line shows a profile in $r^{-1.5}$. For the bottom panel, the dashed-dotted line follows a profile in $r^{-1}$. _Right:_ cross-section as a function of the grain size $s$. The different colors show the cross-section measured in several concentric rings. (*fig:diag*)

</div>
<div id="div_fig3">

<img src="tmp_2406.02682/./HD157587_pfunc.png" alt="Fig1" width="100%"/>

**Figure 1. -** Observations and best fit model for HD 157587 (orange circles and solid black line, respectively). _Top:_ degree of polarization. _Bottom:_ total intensity phase function. The inner and outer radii for the stellocentric distances are indicated in the upper right of the bottom panel. (*fig:HD157*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.02682"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thefigure}{A.\arabic{figure}}$
$\newcommand{\thetable}{A.\arabic{table}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# What excites the optical emission in X-ray-selected galaxies? $\thanks{The catalogue of X-ray galaxies selected from SDSS is only available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via \href{http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}{http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.03254-b31b1b.svg)](https://arxiv.org/abs/2406.03254)<mark>Appeared on: 2024-06-06</mark> -  _12 pages, 13 figures_

</div>
<div id="authors">

<mark>N. Pulatova</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>R. Seeburger</mark>, <mark>M. Demianenko</mark>

</div>
<div id="abstract">

**Abstract:** We present a study of $1347$ galaxies at $z<0.35$ with detected nuclear X-ray emission and optical emission line diagnostics in the Baldwin-Phillips-Terlevich (BPT) diagram. This sample was obtained by cross-matching the X-ray Multi-Mirror Mission Observatory - Newton (XMM-Newton) DR10 catalogue with Sloan Digital Sky Survey (SDSS) DR17 galaxies with well-measured line ratios.  The distribution of these sources in the BPT diagram covers all three excitation regimes: Ionized Hydrogen (HII) regions  (23 \% ), `composites' (30 \% ), and Seyfert galaxies with  the low ionization nuclear emission line regions (LINERs) (47 \% ). In contrast, the fraction of objects classified as active galacticnuclei (AGN) in the SDSS subsample selected for cross-match with XMM-Newton is only 13 \% . This fact illustrates that X-ray emission from galaxies commonly points towards the presence of AGN. Our data show, for the first time, a clear dependence of the BPT position on the ratio of the X-ray to $H\alpha$ fluxes. Sources dominated by X-ray emission lie in the Seyfert and LINER regimes of the BPT diagram. Most sources with a low X-ray-to- $H\alpha$ -luminosity ratio, $log_{10}(L_X/L_{H\alpha}) < 1.0$ , lie in the HII regime. In our sample, there are even 45 galaxies  that have $L^{Star}_{XR}/L^{Total}_{Xray}>0.5$ . In contrast, the positions of  the samplemembers in the BPT diagram exhibit ${no}$ dependence on the X-ray hardness ratio. Our finding suggests that the X-ray-to- $H\alpha$ ratio can help us to differentiate galaxies whose X-ray flux is dominated by an AGN ${from galaxies with}$ central X-ray binaries and other stellar X-ray sources.

</div>

<div id="div_fig1">

<img src="tmp_2406.03254/./DR-17-X-ray-H_alpha_Lum.jpg" alt="Fig5" width="100%"/>

**Figure 5. -** Positions of 1347 SDSS X-ray selected galaxies on X-ray versus $H\alpha$ luminosities plane. X-ray luminosity is in the range 0.2-12 keV and in $erg\cdot s^{-1}$, $H\alpha$ luminosity is in $erg\cdot s^{-1}\cdot$. Type of ionisation (AGNs, Composite, SF) corresponds to the position of X-ray galaxy on [OIII]/$H\beta$ versus [NII]/$H\alpha$ BPT diagram. AGNs are marked with blue circles, Composites - with orange crosses, and SF - with green squares. (*fig:X-ray-H-al*)

</div>
<div id="div_fig2">

<img src="tmp_2406.03254/./DR-17-BPT_sdss-z-X-Ha-1.jpg" alt="Fig9" width="100%"/>

**Figure 9. -** Distribution on the BPT diagram  ([Baldwin, Phillips and Terlevich 1981]())  of $1347$ SDSS X-ray-selected galaxies. Empirical lines (dashed blue and solid red) that separate LINERs from Seyfert galaxies are from [Kewley, et. al (2006)](). The grey background circles show all the $204 895$ SDSS-DR17 galaxies that satisfy our basic optical quality cuts: all line fluxes have fractional uncertainties of $<0.13$ dex (33\%) and are `narrow' $\sigma_{line}<200 km/s$. The colour bar represents X-ray/H$\alpha$ flux ratio (6a) and redshift z (6b) and corresponds only to $1347$ SDSS X-ray-selected galaxies.  (*fig:BPT-DR17*)

</div>
<div id="div_fig3">

<img src="tmp_2406.03254/./DR17-X-ray-RA-DEC.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of the sample of 1347 X-ray-selected galaxies in equatorial coordinates. (*fig:RA-DE-disz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.03254"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\f}{\frac}$
$\newcommand{\tx}{\text}$
$\newcommand{\Li}{\mathcal{L}}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Simultaneous retrieval of orbital phase resolved _JWST_/MIRI emission spectra of the hot Jupiter WASP-43b: evidence of water, ammonia and carbon monoxide

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.03490-b31b1b.svg)](https://arxiv.org/abs/2406.03490)<mark>Appeared on: 2024-06-06</mark> -  _17 pages, 14 figures, accepted for publication in MNRAS. Comments welcome!_

</div>
<div id="authors">

J. Yang, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Spectroscopic phase curves of hot Jupiters measure their emission spectra at multiple orbital phases, thus enabling detailed characterisation of their atmospheres.Precise constraints on the atmospheric composition of these exoplanets offer insights into their formation and evolution.We analyse four phase-resolved emission spectra of the hot Jupiter WASP-43b, generated from a phase curve observed with the MIRI/LRS onboard the _JWST_ , to retrieve its atmospheric properties.Using a parametric 2D temperature model and assuming a chemically homogeneous atmosphere within the observed pressure region, we simultaneously fit the four spectra to constrain the abundances of atmospheric constituents, thereby yielding more precise constraints than previous work that analysed each spectrum independently.Our analysis reveals statistically significant evidence of $NH_3$ (4 $\sigma$ ) in a hot Jupiter's emission spectra for the first time, along with evidence of $H_2$ O (6.5 $\sigma$ ), CO (3.1 $\sigma$ ), and a non-detection of $CH_4$ .With our abundance constraints, we tentatively estimate the metallicity of WASP-43b at 0.6 $-$ 6.5 $\times$ solar and its C/O ratio at 0.6 $-$ 0.9.Our findings offer vital insights into the atmospheric conditions and formation history of WASP-43b by simultaneously constraining the abundances of carbon, oxygen, and nitrogen-bearing species.

</div>

<div id="div_fig1">

<img src="tmp_2406.03490/./Figures/compare_spectra_opacity_1500K_revised.png" alt="Fig11" width="100%"/>

**Figure 11. -** a) - d): Best-fit model spectra at the four orbital phases calculated from the maximum a posteriori parameters.
We plot the 1$\sigma$ and 2$\sigma$ central credible intervals with dark and light purple shading, respectively.
The data in the grey shaded region (10.5-12 $\micron$) are affected by the `shadowed region' systematics and are not included in our retrievals (see \ref{sec:shadow_effect}).
Interestingly, our best-fit model, when extended to 12 $\micron$, is consistent with the current reduction of the `shadowed region' data for all phases except at phase $0.75$(see \ref{sec:spectral_fit} for why we think phase $0.75$ is worst affected by the `shadowed region effect').
Since $NH_3$ has strong spectral features in the `shadowed region', a reliable reduction of the `shadowed region effect' in the future can refine the abundance constraints on $NH_3$.
Note the absorption feature at the 8.75 $\micron$ bin at phase $0.5$, which our model cannot explain.
e) - f): Cross sections of the spectrally active molecules included in our retrievals, computed at a spectral resolution of R$=1000$ at 1 bar pressure and 1500 K temperature.
f) is identical to e) except for the omission of legend for ease of reference.
The spectral features between $\sim$5 and $\sim$8 $\micron$ are indicative of $H_2$O molecules, whereas the absence of spectral features between $\sim$7 and $\sim$9 $\micron$ allows us to rule out $CH_4$ at high abundance.
While $NH_3$ has strong opacities throughout the MIRI/LRS bandpass, its retrieved abundance is about two orders of magnitude lower than $H_2$O.
This means that the spectral features of $H_2$O would dominate over those of $NH_3$ in the wavelength regions where $H_2$O has higher or similar opacity compared to $NH_3$.
The notable exception is the wavelength region greater than $\sim$8 $\micron$, where the $H_2$O opacity is generally much lower than the $NH_3$ opacity.
The constraints on CO are driven by the 5.25 $\micron$  bin; the absence of CO would increase the planetary flux at the 5.25 $\micron$  bin relative to the 5.75 $\micron$ bin.
The constraints on $CO_2$ are driven by the data in 9-10.5 $\micron$ region. (*fig:spectral_fit*)

</div>
<div id="div_fig2">

<img src="tmp_2406.03490/./Figures/model.png" alt="Fig1" width="100%"/>

**Figure 1. -** Schematics of our parametric 2D temperature model as defined by equation (\ref{eq:Tmap}).
The model divides the atmosphere into a dayside region and a nightside region, each modelled with a representative TP profile.
The dayside central longitude and the dayside width are allowed to vary.
`O' marks the substellar point and `O$^{'}$' marks the centre of the dayside region.
While temperature is constant with longitude on isobars in the nightside region, we can parameterise the variation of temperature with longitude on the dayside.
Note that the temperature is constant with latitude and only varies with pressure and longitude (we interpret the retrieved thermal structure as a latitudinal average, see \ref{sec:model}).
 (*fig:model*)

</div>
<div id="div_fig3">

<img src="tmp_2406.03490/./Figures/TP_model_revised.png" alt="Fig6" width="100%"/>

**Figure 6. -** Best-fit 2D thermal structure calculated from the maximum a posteriori parameters.
The white vertical dashed line marks the position of the hot spot offset.
 (*fig:tp_2D*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.03490"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

87  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
